In [123]:
import pandas as pd
import psycopg2 as ps 
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder


### Conexão com o banco de dados

In [124]:
try:
    dbname_extracao = "defaultdb"
    user_extracao = "avnadmin"
    host_extracao = "pg-13f37248-germinare-629e.g.aivencloud.com"
    password_extracao = "AVNS_aO3d1bNC3QQ01jidvM5"
    port_extracao = "17444"
    
    conn = ps.connect(
        dbname=dbname_extracao,
        user=user_extracao,
        host=host_extracao,
        password=password_extracao,
        port=port_extracao
    )
    print("Conexão com o banco de dados estabelecida com sucesso!")
except ps.DatabaseError as e:
    print(f"Erro ao conectar com o banco de dados: {e}")
finally:
    if 'conn' in locals():
        conn.close()
        print("Conexão fechada.")

Conexão com o banco de dados estabelecida com sucesso!
Conexão fechada.


### Extraindo dados do database do 1° ano

In [125]:
# Definindo os parametros para a conexão
dbname_extracao = "defaultdb"
user_extracao = "avnadmin"
host_extracao = "pg-13f37248-germinare-629e.g.aivencloud.com"
password_extracao = "AVNS_aO3d1bNC3QQ01jidvM5"
port_extracao = "17444"
    
conn_params = ps.connect(
    dbname=dbname_extracao,
    user=user_extracao,
    host=host_extracao,
    password=password_extracao,
    port=port_extracao
)
# Estabelecendo a conexão
try:
    conn = ps.connect(**conn_params)
    # Lendo a tabela
    df = pd.read_sql("SELECT * FROM Address", conn)
    #Print para teste
    print(df.head())
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
finally:
    #Fechando a conexão
    conn.close()


C:\Users\daviichihara-ieg\AppData\Local\Temp\ipykernel_36004\1442900572.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM Address", conn)


        state         city        country
0  California  Los Angeles  United States
1       Texas      Houston  United States
2     Ontario      Toronto         Canada
3  Queensland     Brisbane      Australia
4   São Paulo    São Paulo         Brazil


### Normalizando os dados

In [126]:
for col in df.columns:
    if df[col].dtype == 'object':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

### Inserindo em outro banco de dados

In [127]:
import pandas as pd
import psycopg2
from psycopg2 import sql

dbname_insercao = "dbtest2_tr63"
user_insercao = "postgres123"
host_insercao = "dpg-cr4a4hjv2p9s73cp3jdg-a.oregon-postgres.render.com"
password_insercao = "nsut0Y7GaKyu5OgtcCRwnB007hfpIjdD"
port_insercao = "5432"

conn_params = {
    "dbname": dbname_insercao,
    "user": user_insercao,
    "host": host_insercao,
    "password": password_insercao,
    "port": port_insercao
}

# Connect to the PostgreSQL database
conn = psycopg2.connect(**conn_params)
cur = conn.cursor()

insert_query = sql.SQL("INSERT INTO Address ({}) VALUES ({})").format(
    sql.SQL(', ').join(map(sql.Identifier, df.columns)),
    sql.SQL(', ').join(sql.Placeholder() * len(df.columns))
)

# Insert rows one by one
for index, row in df.iterrows():
    cur.execute(insert_query, tuple(row))

# Commit the transaction
conn.commit()

# Close the cursor and connection
cur.close()
conn.close()


### Verificando a inserção dos dados

In [128]:
# Definindo os parametros para a conexão
dbname_insercao = "dbtest2_tr63"
user_insercao = "postgres123"
host_insercao = "dpg-cr4a4hjv2p9s73cp3jdg-a.oregon-postgres.render.com"
password_insercao = "nsut0Y7GaKyu5OgtcCRwnB007hfpIjdD"
port_insercao = "5432"

conn_params = {
    "dbname": dbname_insercao,
    "user": user_insercao,
    "host": host_insercao,
    "password": password_insercao,
    "port": port_insercao
}

# Estabelecendo a conexão
try:
    conn = ps.connect(**conn_params)
    # Lendo a tabela
    df = pd.read_sql("SELECT * FROM Address", conn)
    #Print para teste
    print(df.head())
except Exception as e:
    print(f"Erro ao carregar dados: {e}")
finally:
    #Fechando a conexão
    conn.close()


C:\Users\daviichihara-ieg\AppData\Local\Temp\ipykernel_36004\2603437846.py:20: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM Address", conn)


   state  city  country
0      1     3        8
1      6     2        8
2      3     9        2
3      4     0        0
4      5     7        1
